In [12]:
from datasets import load_dataset
from transformers import AutoTokenizer


In [2]:
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.72M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikitext downloaded and prepared to /Users/theostenlieb/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20. Subsequent calls will reuse this data.


In [4]:
len(datasets["train"][10])

1

In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(datasets["train"])

,text
0,716th Static Infantry Division under Generalleutnant Wilhelm Richter726th Infantry Regiment : two battalions in and around Le Hamel . Members of the 441st Ost Battalion garrisoned the coastal defenses . One battalion was stationed near Crépon as reserves . \n
1,"García Márquez comments on the nature of historical fact by drawing attention to the way history is written . The novel recreates a time in Bolívar 's life that has no historical precedent , as there is no record of the last 14 days of his life . In García Márquez 's account readers observe Bolívar intimately , seeing his human qualities . In the view of critic Isabel Alvarez Borland , by choosing to fictionalize a national hero in this way , García Márquez is challenging the claim of official history to represent the truth . In the "" My Thanks "" section of the novel , García Márquez asserts ironically that what he is writing is more historical than fictional , and he discusses his own historical methodology in detail . By posing in the role of a historian , he challenges the reliability of written history from within the writing process . According to Alvarez Borland , this serves to "" remind us that a claim to truth is not the property of any text ; rather it is the result of how a historian ( as a reader ) interprets the facts "" . \n"
2,"His students included Carol Brooks Gardner , Charles Goodwin , Marjorie Goodwin , John Lofland , Gary Marx , Harvey Sacks , Emanuel Schegloff , David Sudnow and Eviatar Zerubavel . \n"
3,"A Confession of Faith to replace the Thirty @-@ Nine Articles was begun in August 1646 . While there is little record of the actions of the Assembly during the writing of the Confession , it is clear that there were significant debates in the Assembly over almost every doctrine found in it . The Confession was printed and sent to Parliament in December . The House of Commons requested scripture citations be added to the Confession , which were provided in April 1648 . Parliament approved the Confession with revisions to the chapters on church censures , synods and councils , and marriage on 20 June 1648 . The General Assembly of the Church of Scotland had already adopted the Confession without revision in 1647 . The restoration of Charles II in 1660 effectively made this legislation a nullity . \n"
4,"By the autumn of 1990 , after the first democratic elections in Croatia , the municipal government finally approved the project . An architectural competition was planned for January 1991 . Political turmoil in the country , followed by the breakup of Yugoslavia and the Croatian War of Independence ( 1991 – 1995 ) , caused the project to be put on hold again . In 1994 President of Croatia Franjo Tuđman said to Jakov Bienenfeld , Council member of the Zagreb Jewish community , that they should build the new synagogue at the site of the former synagogue , which will be funded by the Croatian government . Bienenfeld declined the offer believing to be inappropriate when 1800 Catholic churches are left destroyed at the time , during Croatian War of Independence . \n"
5,"At its peak in 1954 , DuMont was affiliated with around 200 television stations . In those days , television stations were free to "" cherry @-@ pick "" which programs they would air , and many stations affiliated with multiple networks , depending mainly on the number of commercial television stations available in a market at a given time ( markets where only one commercial station was available carried programming from all four major networks ) . Many of DuMont 's "" affiliates "" carried very little DuMont programming , choosing to air one or two more popular programs ( such as Life Is Worth Living ) and / or sports programming on the weekends . Few stations carried the full DuMont program lineup . \n"
6,
7,
8,= = = Season standings = = = \n
9,


In [13]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)


Downloading:   0%|          | 0.00/236 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/678k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [14]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [15]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])


In [16]:
tokenized_datasets["train"][1]


{'attention_mask': [1, 1, 1, 1, 1, 1],
 'input_ids': [238, 8576, 9441, 2987, 238, 252]}

In [17]:
tokenizer.model_max_length

1024

In [19]:
sum([1,2,3], [])

TypeError: can only concatenate list (not "int") to list

In [18]:
block_size = 128


In [21]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [22]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [24]:
tokenizer.decode(lm_datasets["train"][2]["input_ids"])


" team of writers handled the script. The game's opening theme was sung by May 'n. \n It met with positive sales in Japan, and was praised by both Japanese and western critics. After release, it received downloadable content, along with an expanded edition in November of that year. It was also adapted into manga and an original video animation series. Due to low sales of Valkyria Chronicles II, Valkyria Chronicles III was not localized, but a fan translation compatible with the game's expanded edition was released in 2014. Media.Vision would return to the franchise with the development of Valkyria : Azure Revolution for the PlayStation 4. \n"

In [25]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_config(config)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [26]:
from transformers import Trainer, TrainingArguments


In [27]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [29]:
trainer.train()


***** Running training *****
  Num examples = 17991
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6747


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 